In [68]:
import pandas as pd
import re

In [69]:
# FILEPATH: /c:/Users/sedad/Documents/master_code/svvr/brein/data.ipynb
position_file_loc = 'data/rank_0_positions.txt'
no_network_file = 'data/rank_0_step_1000000_out_no_network.txt'

step = re.search(r'step_(\d+)', no_network_file).group(1)
step = int(step)

In [70]:
df_no_network = pd.read_csv(no_network_file, delimiter=r'\s+', skiprows=8, names=['target_rank', 'target_id', 'source_rank', 'source_id', 'weight'])
df_positions = pd.read_csv(position_file_loc, delimiter=r'\s+', skiprows=8, names=['localid', 'posx', 'posy', 'posz', 'area', 'type'])
df_positions['area'] = df_positions['area'].str.replace('area_', '')
df_positions.drop('type', axis=1, inplace=True)

In [71]:
df_positions

,localid,posx,posy,posz,area
0,1,88.017654,143.889110,83.259131,43
1,2,88.071787,143.792200,83.237149,43
2,3,88.093434,143.951900,83.168331,43
3,4,88.063898,143.819810,83.317563,43
4,5,88.066131,143.973540,83.238470,43
...,...,...,...,...,...
49995,49996,86.890937,1.931412,79.944581,3
49996,49997,86.764390,1.834104,80.037049,3
49997,49998,86.699561,1.970323,79.967185,3
49998,49999,86.841367,1.944717,80.029320,3


In [72]:
df_no_network

,target_rank,target_id,source_rank,source_id,weight
0,0,572,0,1,1
1,0,47,0,1,1
2,0,516,0,1,1
3,0,1422,0,1,1
4,0,739,0,1,1
...,...,...,...,...,...
720084,0,47062,0,50000,1
720085,0,48923,0,50000,1
720086,0,49805,0,50000,1
720087,0,48964,0,50000,1


In [73]:
# Find the x, y, z, and area of the target neurons
df1 = pd.merge(df_no_network, df_positions[['localid', 'posx', 'posy', 'posz', 'area']], left_on='target_id', right_on='localid', how='left')
df1.drop('localid', axis=1, inplace=True)
df1.rename(columns={'posx': 'target_posx', 'posy': 'target_posy', 'posz': 'target_posz', 'area': 'target_area'}, inplace=True)

# Find the x, y, z, and area of the source neurons
df2 = pd.merge(df_no_network, df_positions[['localid', 'posx', 'posy', 'posz', 'area']], left_on='source_id', right_on='localid', how='left')
df2.drop('localid', axis=1, inplace=True)
df2.rename(columns={'posx': 'source_posx', 'posy': 'source_posy', 'posz': 'source_posz', 'area': 'source_area'}, inplace=True)

# Merge the two dataframes
df = pd.concat([df1, df2], axis=1)
df.drop(['target_rank', 'source_rank'], axis=1, inplace=True)
df = df.loc[:, ~df.columns.duplicated()]
df

,target_id,source_id,weight,target_posx,target_posy,target_posz,target_area,source_posx,source_posy,source_posz,source_area
0,572,1,1,73.214799,143.973190,88.638902,9,88.017654,143.889110,83.259131,43
1,47,1,1,85.646000,143.891880,87.027818,43,88.017654,143.889110,83.259131,43
2,516,1,1,90.333513,141.632950,97.605433,43,88.017654,143.889110,83.259131,43
3,1422,1,1,109.479720,137.726160,79.683010,30,88.017654,143.889110,83.259131,43
4,739,1,1,103.749030,140.773430,86.051326,30,88.017654,143.889110,83.259131,43
...,...,...,...,...,...,...,...,...,...,...,...
720084,47062,50000,1,103.647550,7.500531,48.899972,10,86.883557,2.006636,79.997225,3
720085,48923,50000,1,92.179167,1.347507,59.602955,10,86.883557,2.006636,79.997225,3
720086,49805,50000,1,85.639196,3.707297,88.987563,3,86.883557,2.006636,79.997225,3
720087,48964,50000,1,104.268570,3.007913,69.124771,10,86.883557,2.006636,79.997225,3


In [74]:
df.to_csv(f'data/positions_no_network_{step}.csv', index=False)